Copyright 2019 Intel Corporation

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Introduction

In Lesson 4, _Anomaly detection: Proximity-based methods_, we looked at ways of flagging anomalies by using a variety of distance-based methods. In particular, we looked at flagging points using
- distance based methods: anomalies are points "far" from other points
- clustering methods: anomalies are points that are at the edges of their cluster / not assigned to a cluster
- density-based methods: anomalies are points with fewer neighbors than typical points in the same region.

# Learning outcomes

You should come away from this notebook with the ability to
1. Make a kNN anomaly detector (distance-based) 
2. Make a k-means anomaly detector (cluster-based) 
3. Use the Local Outlier Factor to classify anomalies 


# Imports

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets as sk_data
import sklearn.neighbors as neighbors
import sys

np.set_printoptions(suppress=True, precision=4)

# Python and library versions

In [ ]:
packages = [np]

msg = f"""
Python Version: {sys.version}

library .      version
-------        -------"""
print(msg)

for package in packages:
    print(f"{package.__name__:11}    {package.__version__:>7}")

# Section 1: Distance-Based Methods

The simplest Nearest Neighbor (NN) method is to take the distance to the k-nearest neighbor as the anomaly score.  The next variation is to use the average of the distances to the top-k neighbors as the score.  `sklearn` provides `neighbors.NearestNeighbors` to compute the nearest neighbors from a dataset.  After fitting, we can use `kneighbors()` to return the distances and indices of the top-k neighbors.  We can use `kneighbors_graph()` to return the entire connectivity graph (as an array with 1 indicating a link) for the dataset.  The graph is a sparse array, but you can operate on it the same as a normal (dense) array.  If you must have a normal array, you can use `todense()` to make it dense.

The idea for the kNN models is 
- look at the $k$ neighbors of each point
- assign a score. Roughly, a low score means the point's $k$ neighbors are close.
- there are multiple ways of determining the score:
    - maximum of the distances (i.e. distance to $k$th nearest neighbor)
    - average the $k$ distances
    - harmonic mean of the $k$ distances.

Use the score to determine if something is an anomaly by using either a _threshold_ or _ranking_.

We'll start by creating a simple dataset with one anomaly and then plot the data. 

In [ ]:
X = np.array([[0.9, 1], [0, 1], [1, 0], [0, 0], [0.5, 0.5], [0.2, 0.5], [1, 0.5], [2, 2]])

plt.figure(dpi=150)
plt.title('Upper right point looks anomalous')
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.plot(X[:, 0], X[:, 1], 'o');

Let's create a function to visualize a query point and its neighbors more explicitly.

In [ ]:
def plot_point_and_k_neighbors(X, highlight_index, n_neighbors=2):
    "Plots the points in X, and shows the n_neighbors of the highlight_index-th point"
    nn = neighbors.NearestNeighbors(n_neighbors=n_neighbors).fit(X)
    dist, index = nn.kneighbors()
    
    src_pt = X[highlight_index, :]
    
    plt.figure(dpi=150)
    # draw lines first, so points go on top
    for dest_index in index[highlight_index]:
        dest_pt = X[dest_index, :]
        plt.plot(*list(zip(src_pt, dest_pt)), 'k--')
    plt.plot(X[:, 0], X[:, 1], 'o', label='Not k-neighbors', alpha=0.3)
    plt.plot(*src_pt, 'o', label='The query point')
    plt.plot(X[index[highlight_index], 0], X[index[highlight_index], 1], 'o', label='k-neighbors')
    plt.xlabel('$X_1$')
    plt.ylabel('$X_2$')
    plt.legend()
    
# Example of usage
plot_point_and_k_neighbors(X, 0, 4)

Here we can see the 4 points close to the query point (index of 0 in our data) are relatively close. Let's look at the anomalous point in the upper right (this point has an index of 7):

In [ ]:
plot_point_and_k_neighbors(X, 7, n_neighbors=4)

We can see in this case all of the 4-nearest neighbors to our point are far away, so it seems reasonably unambiguous to call this point an anomaly.

Let's add one more point next to the anomaly, and see how choosing "k" and the scoring function affects things.

In [ ]:
# Toy dataset with two adjacent anomalies
X2 = np.concatenate([X, [[1.9, 2.0]]])

# Look at nearest neighbor (k=1)
plot_point_and_k_neighbors(X2, 7, n_neighbors=1)

### Observation 1: dependence on $k$

The distance between point 7 and the new point is actually quite small—smaller than the distances between any pair of the points in the lower left. If $k=1$, the two points in the upper right would be the _last_ ones to classified as an anomaly (i.e. they would have the lowest score).

### Observation 2: how to weight distances?

Let's look at the same data set (i.e. with the additional point) but now look at $k=3$

In [ ]:
plot_point_and_k_neighbors(X2, 7, n_neighbors=3)

We see one of the 3 nearest neighbors is very close, but the other two are far away. We need some way of combining these 3 distances into a single score. The three commonly used methods are
1. Use the longest distance
2. Use the (arithmetic) mean distance
3. Use the harmonic mean.

We will implement the longest distance, and assign the arithmetic mean distance and harmonic mean distance as exercises.

### Using the longest distance

In [ ]:
def do_nn_outlier_scores(obs, n_neighbors=1):
    """
    Gives the score of a point as the distance from point to its k-th nearest neighbor.
    Larger score means more likely to be an outlier
    """
    nn = neighbors.NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(obs)
    dists, idx = nn.kneighbors()
    scores = dists[:,-1]
    return scores

# Test 
do_nn_outlier_scores(X2, n_neighbors=3)

We can make the output a little easier to understand

In [ ]:
def print_ranked_scores(obs, scores):
    scores_and_obs = sorted(zip(scores, obs), key=lambda t: t[0], reverse=True)
    print('Rank  Point\t\tScore')
    print('------------------------------')
    for index, score_ob in enumerate(scores_and_obs):
        score, point = score_ob
        print(f'{index+1:3d}.  {point}\t\t{score:6.4f}')

# Look at the outliers using 3 neighbors
print_ranked_scores(X2, do_nn_outlier_scores(X2, 3))

Now look at the anomalies with k=1

In [ ]:

print_ranked_scores(X2, do_nn_outlier_scores(X2, 1))

**TAKEAWAY:** The number of neighbors $k$ used in kNN drastically changes the results. Usually requires some domain expertise to know the value of $k$ or range of values for $k$ to be used.

## kNN in reverse: outlier detection using in-degree (ODIN)


Another distance-based variation, called ODIN (outlier detection using in-degree), uses the same neighbors in a different way.  As discussed in the lesson, this algorithm asks the question, "who am I the nearest neighbor of?".  So, an example that serves as a NN of many others -- with many incoming links in the NN graph -- has a *low* anomaly score.  Examples with a small in-degree (= # of incoming links), are more likely to be anomalies. 

Here's an implementation of ODIN. Note that we convert the scores from the indegree value (low for an anomaly) to a zero-to-one scale where higher values are more of anomaly.

In [ ]:
def do_odin_outlier_scores(obs, n_neighbors=3):
    nn = neighbors.NearestNeighbors(n_neighbors=n_neighbors).fit(obs)
    graph = nn.kneighbors_graph()
    indegree = graph.sum(axis=0)  # sparse matrix

    # smaller indegree means more of an anomaly  
    # simple conversion to [0,1] so larger score is more of anomaly
    scores = (indegree.max() - indegree) / indegree.max()
    return np.array(scores)[0]  # convert to array

Score and rank the points for the _X2_ dataset (two adjacent anomalies).

In [ ]:
scores_odin= do_odin_outlier_scores(X2)

In [ ]:
print_ranked_scores(X2, scores_odin)

The two anomalies (2, 2) and (1.9, 2) have large scores, but so do the points (0, 1) and (0, 1). Also, the result is different from we found with k-means results. As we discussed in the lecture, the method you use matters.

# Section 2: Cluster-Based Method (k-Means)

Clustering via k-means can be used as an anomaly detection method by:

  1. generating clusters, 
  2. finding the cluster of each example, 
  3. using the distance from the example to its cluster's center as a score
  
These pieces are available using `cluster.KMeans` which after fitting and predicting to get the clusters -- both where the clusters are and what cluster an example belongs to -- provides a `cluster_centers_` attribute.  Then we can use numpy's `numpy.linalg.norm` (often imported as `import numpy.linalg as nla`) to compute distances for us.

We will start by generating some data, then showing what the clusters are, and finally scoring the anomalies. 

**Data generation and view**

In [ ]:
blobs_X, cluster_labels = sk_data.make_blobs(centers=[[0,0], [10,10], [10,0]])
anomalies, _ = sk_data.make_blobs(centers=[[5,5]], n_samples=5, cluster_std=3, random_state=42)

data = np.concatenate([blobs_X, anomalies])
cluster_labels = np.concatenate([cluster_labels, [-1]*len(anomalies)])

# Display the data before clustering
plt.figure(dpi=110)
plt.plot(data[:, 0], data[:, 1], 'o');

**Clustering and show cluster assignment**

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3).fit(data)

plt.figure(dpi=120)

for label in range(3):
    mask = (km.labels_ == label)
    plt.plot(data[mask, 0], data[mask, 1], 'o', label=f'Cluster {label}')
plt.legend();

You should run the clustering several times and note that the assignment of some points changes due to the different initial conditions (random choice of starting centroids).

**For each point, find the distance from the point to its centroid**

In [ ]:
centers = km.cluster_centers_[km.labels_]
# show the centers for the first 10 points
centers[:10]

In [ ]:
#Get the distances to the centers as use these as the scores
score = np.linalg.norm(data - centers, axis=1)
score

What are the 5 points with the highest scores? We will call those the anomalies

In [ ]:

np.argsort(score)[::-1][:5]

Show these results in a plot that includes the cluster centers (labeled with an 'x')

In [ ]:

from sklearn.cluster import KMeans

km = KMeans(n_clusters=3).fit(data)

anomaly_idx = np.argsort(score)[::-1][:5]
anomaly_mask = np.zeros(len(data))
anomaly_mask[anomaly_idx] = 1

plt.figure(dpi=120)
colors = ['blue', 'orange', 'green']

for label, color in enumerate(colors):
    mask = (km.labels_ == label) & (anomaly_mask == 0)
    plt.plot(data[mask, 0], data[mask, 1], marker='o', linestyle='none',
             color=color, label=f'Cluster {label}')
    plt.plot(*km.cluster_centers_[label], marker='x', color='k')

plt.plot(data[anomaly_idx, 0], data[anomaly_idx, 1], marker='o', linestyle='none',
         color='k', label='Anomaly')
plt.legend();

If you rerun the clustering enough times, you will find that the three central points are always labeled as anomalies. The other two anomalies vary depending on how the points are clustered.

**Note**: even bigger changes can be expected if the number of clusters is varied. Below we illustrate the effect of varying the number of clusters from 1 to 5.

In [ ]:
# Show clustering for user-specified number of clusters
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1,5, figsize=(15,3))
for ax, n_clusters in zip(axes, [1,2,3,4,5]):
    clust = KMeans(n_clusters=n_clusters)
    obs_to_clusters = clust.fit_predict(data) 
    ax.scatter(*data.T, c=obs_to_clusters)
    ax.set_title("n_clusters={}".format(n_clusters))

# Section 3: Density-Based Methods (Local Outlier Factor)

The local outlier factor (LOF) is `sklearn` has a version available for us to use. Note the the LOF returned by `sklearn` is the negative of the value we defined in the lecture.

Apply LOF to the collection of points ('X2') we created in section 1.

In [ ]:
import sklearn.neighbors as neighbors
lof = neighbors.LocalOutlierFactor(n_neighbors=3, contamination='auto')
lof.fit(X2)
sk_lof = -lof.negative_outlier_factor_
print_ranked_scores(X2, sk_lof)

See values both above and below 1. As discussed in the lecture, we expect the values significantly above 1 to be anomalies.

Compare with the kNN results from before. Note we use the same value of k (=3) for both algorithms.

In [ ]:
print_ranked_scores(X2, do_nn_outlier_scores(X2, 3))

The two anomalous points are both found either way, but there are differences between the results for the other points. 

For completeness, we provide below an explicit implementation of LOF that follows the outline given in the lecture. To check that our algorithm is reasonable, we compare the results for its anomaly score with those of the `sklearn` version for the cluster blobs ('data').

In [ ]:
def lof_method(obs, n_neighbors=2):
    neigh = neighbors.NearestNeighbors(n_neighbors=2).fit(obs)
    
    #Return indices of and distances to the neighbors of each point
    topk_dist, my_kneigh = neigh.kneighbors()
    
    # Create list of distances of furthest (kth) neighbor 
    k_dist = topk_dist[:,-1]
 
    # Reachability distance: maximum of true distance between query neighbor and query point
    # and distance to kth nearest neighbor of query neighbor
    reach = np.maximum(topk_dist, k_dist[my_kneigh])
    
    # Local reacability density is reciprocal of average reachability distance
    lrd = 1.0 / np.mean(reach, axis=1)
    
    # Local outlier factor is given by
    # average local density of neighbors / local density of query point
    lrd_ratios = lrd[my_kneigh] / lrd[:, np.newaxis]
    lof = np.mean(lrd_ratios, axis=1)
    
    return lof

In [ ]:
# Choose k=2
our_lof = lof_method(data, 2)
our_lof

In [ ]:
lof2 = neighbors.LocalOutlierFactor(n_neighbors=2, contamination='auto')
lof2.fit(data)
sk_lof2 = -lof2.negative_outlier_factor_
print('Same as sklearn?', np.allclose(our_lof, sk_lof2))

### Exercise #1

This exercise refers to the distance-based methods (Section 1).

A. Create a function `do_nn_avg_scores(obs, n_neighbors=1)` that computes outlier scores using arithmetic mean distance from the point to each of the `n_neighbors` nearest neighbors as the score. 

B. Do the same thing as in part (A) to create `do_nn_harm_scores(obs, n_neighbors=1)`, where you use the harmonic mean instead of the mean. The harmonic mean of $n$ points is defined as

$$\text{harmonic}(X_1, X_2, \ldots, X_n) = \frac{n}{(1/x_1) + (1/x_2) + \ldots + (1/x_n)} = \frac{\left(\prod X_i\right)^{1/n}}{\bar{X}} = \frac{(X_1X_2\cdot X_n)^{1/n}}{\bar{X}}$$

   Note that `scipy.stats` contains a `hmean` function you can use.
  
Try each of these functions on our toy dataset, $X2$.

### Solution #1

In [ ]:
# YOUR CODE HERE

### Exercise #2

This exercise refers to cluster-based methods (k-means; Section 2).

A. Combine the code in the individual cells to create a function `do_cluster_outlier_scores(obs, n_clusters=3)` that computes outlier scores using the cluster method. Check that you find the same top five anomalies as before.

B. Combine the code in the individual cells to create a function `plot_clusters_and_outliers(obs, n_clusters=3, n_anomalies=5)` that generates a plot like the one above. Do you get the same results as before?

### Solution #2

In [ ]:
# YOUR CODE HERE

### Exercise #3

This exercise refers to density-based methods (Local Outlier Factor; Section 3).


A. Create a function `do_lof_outlier_scores(obs, n_neighbors=3)` that computes outlier scores using the LOF method.  Recall that the values returned by sklearn's implementation are negatives of what we want.

B. Apply your function to our toy dataset `data` to get the top five anomalies.

C. How sensitive is the LOF method to `n_neighbors`?  Trying varying `n_neighbors` on the `data` dataset.

### Solution #3

In [ ]:
# YOUR CODE HERE

# Summary

In this assignment you should have learned: 

1. How to make a kNN anomaly detector (distance based) 
2. How to make a k-means anomaly detector (cluster based) 
3. To use the Local Outlier Factor to classify anomalies 

Congratulations! This concludes the lesson.